# Notebook 0: Setup LFP trial dataframe

Purpose:
---------
This notebook is dedicated to extracting local field potential (LFP) traces from Spikegadgets .rec files 
for the purpose of analyzing social competition trials in neuroscience research. The notebook processes 
the raw electrophysiology data, synchronizes it with corresponding video data, and computes various metrics.

Dataframe Columns Description:
------------------------------
- time: Spikegadgets timestamp used during recording.
- state: The state of the environmental control unit's (ECU) input for Spikegadgets recordings.
- recording_dir: Directory path where the recording file is located.
- recording_file: Base name of the recording file without the extension.
- din: Identifier of the input from the Spikegadgets ECU.
- time_stamp_index: Index representing the specific timestamp of the recording.
- video_file: Basename of the associated video file for the trial.
- video_frame: Specific video frame number corresponding to the trial.
- video_number: Unique number identifying the video associated with the trial.
- subject_info: Information about the subject, such as ID, age, or other relevant details.
- condition: Label categorizing the trial, typically based on experimental conditions.
- competition_closeness: Secondary label detailing the competitive closeness of the trial.
- lfp_index: Index associated with the LFP data for the trial.
- video_name: Basename of the video file associated with the LFP recording.
- baseline_lfp_index_range: Range of LFP indices representing the baseline period of the trial.
- trial_lfp_index_range: Range of LFP indices corresponding to the actual trial period.
- baseline_ephys_index_range: Range of electrophysiology (ephys) indices representing the baseline period.
- trial_ephys_index_range: Range of ephys indices corresponding to the trial period.
- baseline_videoframe_range: Range of video frames representing the baseline period of the trial.
- trial_videoframe_range: Range of video frames corresponding to the trial period.
- all_subjects: List or array of all subjects involved in the trials.
- current_subject: ID number or identifier of the subject for the current row in the DataFrame.
- trial_outcome: Updated trial label based on the "condition" column.

Processing Steps:
-----------------
1. Importing necessary libraries and setting up the environment.
2. Loading and preprocessing the tone timestamp data from an Excel file.
3. Identifying and processing all session directories containing .rec files.
4. Reformatting the DataFrame for analysis, including dropping unnecessary rows and columns.
5. Adding columns for different timestamps and ranges (LFP, ephys, video frames).
6. Converting trial labels to 'win' or 'lose' based on trial outcomes.
7. Adding a column for competition closeness and reformatting it.
8. Saving the processed DataFrame to a specified output directory.

Outputs:
--------
- Processed DataFrame saved as a CSV file, containing synchronized and labeled LFP and video data.
- Plots or other outputs can be added as per the project's specific analysis requirements.

Usage:
------
- Run the cells in order as they appear.
- Ensure that the file paths and directory names match your project's structure.
- Customize processing steps according to your project's specific needs.

Note:
-----
- This notebook is part of a larger research project. Ensure compatibility with other components of the project.
- The notebook is configured for a specific dataset structure and might require modifications for different data formats.

In [1]:
import sys
import os
import git

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/projects/reward_competition_extention'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
# Imports of all used packages and libraries
import glob
import numpy as np
import pandas as pd

In [6]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

In [7]:
from utilities import helper

## Inputs & Data

Explanation of each input and where it comes from.

In [8]:
EPHYS_SAMPLING_RATE = 20000
LFP_SAMPLING_RATE = 1000
TRIAL_DURATION = 10
FRAME_RATE = 22
ECU_STREAM_ID = "ECU"
TRODES_STREAM_ID = "trodes"
LFP_FREQ_MIN = 0.5
LFP_FREQ_MAX = 300
ELECTRIC_NOISE_FREQ = 60
RECORDING_EXTENTION = "*.rec"

In [9]:
EPHYS_INDEX_COL = "time_stamp_index"
LFP_INDEX_COL = "lfp_index"
EPHYS_TIMESTAMP_COL = "time"

In [10]:
ORIGINAL_TRIAL_COL = "condition"
TRIAL_OUTCOME_COL = "trial_outcome"
RECORDING_DIR_COL = "recording_dir"
VIDEO_FRAME_COL = "video_frame"

In [11]:
STATE_COL = "state"
RECORDING_FILE_COL = "recording_file"
DIN_COL = "din"
TIME_STAMP_INDEX_COL = "time_stamp_index"
VIDEO_FILE_COL = "video_file"
VIDEO_NUMBER_COL = "video_number"
SUBJECT_INFO_COL = "subject_info"
COMPETITION_CLOSENESS_COL = "competition_closeness"
VIDEO_NAME_COL = "video_name"
BASELINE_LFP_INDEX_RANGE_COL = "baseline_lfp_index_range"
TRIAL_LFP_INDEX_RANGE_COL = "trial_lfp_index_range"
BASELINE_EPHYS_INDEX_RANGE_COL = "baseline_ephys_index_range"
TRIAL_EPHYS_INDEX_RANGE_COL = "trial_ephys_index_range"
BASELINE_VIDEOFRAME_RANGE_COL = "baseline_videoframe_range"
TRIAL_VIDEOFRAME_RANGE_COL = "trial_videoframe_range"
ALL_SUBJECTS_COL = "all_subjects"
CURRENT_SUBJECT_COL = "current_subject"


In [12]:
# NOTE: Change based on individual project data location

# Spreadsheet of tone time
TONE_TIMESTAMP_DF = pd.read_excel("../../data/rce_tone_timestamp.xlsx", index_col=0)

In [13]:
TONE_TIMESTAMP_DF.head()

,time,state,recording_dir,recording_file,din,time_stamp_index,video_file,video_frame,video_number,subject_info,condition,competition_closeness,Unnamed: 13
2299.0,3772337.0,1.0,20221122_161341_omission_subject_6_1_and_6_3,20221122_161341_omission_subject_6_1_top_4_base_2,dio_ECU_Din1,0.0,20221122_161341_omission_subject_6_1_and_6_3.1...,0.0,1.0,6_1_top_4_base_2,NaN,NaN,NaN
2300.0,5204112.0,1.0,20221122_161341_omission_subject_6_1_and_6_3,20221122_161341_omission_subject_6_1_top_4_base_2,dio_ECU_Din1,1431775.0,20221122_161341_omission_subject_6_1_and_6_3.1...,1784.0,1.0,6_1_top_4_base_2,NaN,NaN,NaN
2301.0,6804107.0,1.0,20221122_161341_omission_subject_6_1_and_6_3,20221122_161341_omission_subject_6_1_top_4_base_2,dio_ECU_Din1,3031770.0,20221122_161341_omission_subject_6_1_and_6_3.1...,3779.0,1.0,6_1_top_4_base_2,NaN,NaN,NaN
2302.0,8604101.0,1.0,20221122_161341_omission_subject_6_1_and_6_3,20221122_161341_omission_subject_6_1_top_4_base_2,dio_ECU_Din1,4831764.0,20221122_161341_omission_subject_6_1_and_6_3.1...,6021.0,1.0,6_1_top_4_base_2,NaN,NaN,NaN
2303.0,10204096.0,1.0,20221122_161341_omission_subject_6_1_and_6_3,20221122_161341_omission_subject_6_1_top_4_base_2,dio_ECU_Din1,6431759.0,20221122_161341_omission_subject_6_1_and_6_3.1...,8015.0,1.0,6_1_top_4_base_2,NaN,NaN,NaN


In [14]:
# NOTE: Change based on individual project data location
# Where all the recording files are being saved
ALL_SESSION_DIR = glob.glob("/scratch/back_up/reward_competition_extention/data/omission/*/*.rec")

In [15]:
ALL_SESSION_DIR

['/scratch/back_up/reward_competition_extention/data/omission/2023_06_17/20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_17/20230617_115641_standard_comp_to_omission_D1_subj_2-2_and_2-4.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2022_12_02/20221202_134600_omission_and_competition_subject_6_1_and_6_2.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_21/20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_20/20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2022_12_03/20221203_154800_omission_and_competition_subject_6_4_and_6_1.rec',
 '/scratch/back_up/reward_competition_extention/data/omission/2023_06_18/20230618_100646_standard_comp_to_omission_D2_subj_2-4_and_2-1.rec',
 '/scratch/back

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [16]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"../../proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [17]:
TONE_TIMESTAMPS_CSV = "rce_tone_timestamps.csv"
TONE_TIMESTAMPS_PKL = "rce_tone_timestamps.pkl"

# Functions

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [18]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




# Reformatting Dataframe

- Dropping all rows that have not been labeled

In [19]:
TONE_TIMESTAMP_DF = TONE_TIMESTAMP_DF.dropna(subset=ORIGINAL_TRIAL_COL).reset_index(drop=True)

In [20]:
sorted(TONE_TIMESTAMP_DF[RECORDING_DIR_COL].unique())

['20221202_134600_omission_and_competition_subject_6_1_and_6_2',
 '20221203_154800_omission_and_competition_subject_6_4_and_6_1',
 '20221214_125409_om_and_comp_6_1_and_6_3',
 '20221215_145401_comp_amd_om_6_1_and_6_3',
 '20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3',
 '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2',
 '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1',
 '20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4',
 '20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1',
 '20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2']

- Adding the LFP index

In [21]:
TONE_TIMESTAMP_DF[EPHYS_INDEX_COL] = TONE_TIMESTAMP_DF[EPHYS_INDEX_COL].astype(int)

In [22]:
TONE_TIMESTAMP_DF[EPHYS_TIMESTAMP_COL] = TONE_TIMESTAMP_DF[EPHYS_TIMESTAMP_COL].astype(int)

In [23]:
TONE_TIMESTAMP_DF[LFP_INDEX_COL] = (TONE_TIMESTAMP_DF[EPHYS_INDEX_COL] // (EPHYS_SAMPLING_RATE/LFP_SAMPLING_RATE)).astype(int)

- Making the video frame number usable

In [24]:
TONE_TIMESTAMP_DF[VIDEO_FRAME_COL] = TONE_TIMESTAMP_DF[VIDEO_FRAME_COL].astype(int)

- Getting the name of the video so that we can sync it up with the ephys recording

In [25]:
TONE_TIMESTAMP_DF[VIDEO_NAME_COL]  = TONE_TIMESTAMP_DF[VIDEO_FILE_COL].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))

- Making columns of the different timestamps

In [26]:
TONE_TIMESTAMP_DF[BASELINE_LFP_INDEX_RANGE_COL] = TONE_TIMESTAMP_DF[LFP_INDEX_COL].apply(lambda x: (x - TRIAL_DURATION * LFP_SAMPLING_RATE, x))

In [27]:
TONE_TIMESTAMP_DF[TRIAL_LFP_INDEX_RANGE_COL] = TONE_TIMESTAMP_DF[LFP_INDEX_COL].apply(lambda x: (x, x + TRIAL_DURATION * LFP_SAMPLING_RATE))

In [28]:
TONE_TIMESTAMP_DF[BASELINE_EPHYS_INDEX_RANGE_COL] = TONE_TIMESTAMP_DF[EPHYS_INDEX_COL].apply(lambda x: (x - TRIAL_DURATION * EPHYS_SAMPLING_RATE, x))

In [29]:
TONE_TIMESTAMP_DF[TRIAL_EPHYS_INDEX_RANGE_COL] = TONE_TIMESTAMP_DF[EPHYS_INDEX_COL].apply(lambda x: (x, x + TRIAL_DURATION * EPHYS_SAMPLING_RATE))

In [30]:
TONE_TIMESTAMP_DF[BASELINE_VIDEOFRAME_RANGE_COL] = TONE_TIMESTAMP_DF[VIDEO_FRAME_COL].apply(lambda x: (x - TRIAL_DURATION * FRAME_RATE, x))

In [31]:
TONE_TIMESTAMP_DF[TRIAL_VIDEOFRAME_RANGE_COL] = TONE_TIMESTAMP_DF[VIDEO_FRAME_COL].apply(lambda x: (x, x + TRIAL_DURATION * FRAME_RATE))

In [32]:
TONE_TIMESTAMP_DF.columns

Index(['time', 'state', 'recording_dir', 'recording_file', 'din',
       'time_stamp_index', 'video_file', 'video_frame', 'video_number',
       'subject_info', 'condition', 'competition_closeness', 'Unnamed: 13',
       'lfp_index', 'video_name', 'baseline_lfp_index_range',
       'trial_lfp_index_range', 'baseline_ephys_index_range',
       'trial_ephys_index_range', 'baseline_videoframe_range',
       'trial_videoframe_range'],
      dtype='object')

In [33]:
TONE_TIMESTAMP_DF = TONE_TIMESTAMP_DF.drop(columns=["state", "din", "Unnamed: 13"], errors="ignore")

In [34]:
TONE_TIMESTAMP_DF.to_csv(os.path.join(OUTPUT_DIR, TONE_TIMESTAMPS_FILE_NAME))

NameError: name 'TONE_TIMESTAMPS_FILE_NAME' is not defined

# Custom processing

- NOTE: Rest of notebook is project specific processing of collected data. Run the cells below if the format is similar to original project. 

In [ ]:
raise ValueError()

- Getting all subject IDs for a given recording

In [35]:
# using different id extractions for different file formats
TONE_TIMESTAMP_DF[ALL_SUBJECTS_COL] = TONE_TIMESTAMP_DF[RECORDING_DIR_COL].apply(lambda x: x if "2023" in x else "subj" + "_".join(x.split("_")[-5:]))
TONE_TIMESTAMP_DF[ALL_SUBJECTS_COL] = TONE_TIMESTAMP_DF[ALL_SUBJECTS_COL].apply(lambda x: tuple(sorted([num.strip("_").replace("_",".") for num in x.replace("-", "_").split("subj")[-1].strip("_").split("and")])))

In [36]:
TONE_TIMESTAMP_DF[ALL_SUBJECTS_COL].unique()

array([('6.1', '6.2'), ('6.1', '6.4'), ('6.1', '6.3'), ('1.3', '1.4'),
       ('1.1', '1.2'), ('1.1', '1.4'), ('1.2', '1.4')], dtype=object)

In [37]:
TONE_TIMESTAMP_DF[CURRENT_SUBJECT_COL] = TONE_TIMESTAMP_DF[SUBJECT_INFO_COL].apply(lambda x: ".".join(x.replace("-","_").split("_")[:2])).astype(str)

In [38]:
TONE_TIMESTAMP_DF[CURRENT_SUBJECT_COL].unique()

array(['6.1', '1.3', '1.4', '1.1', '1.2'], dtype=object)

- Converting the trial label to win or lose based on who won the trial

In [39]:
TONE_TIMESTAMP_DF[TRIAL_OUTCOME_COL] = TONE_TIMESTAMP_DF.apply(
    lambda x: "win" if str(x[ORIGINAL_TRIAL_COL]).strip() == str(x[CURRENT_SUBJECT_COL]) 
             else ("lose" if str(x[ORIGINAL_TRIAL_COL]) in x[ALL_SUBJECTS_COL] 
                   else x[ORIGINAL_TRIAL_COL]), axis=1)

In [40]:
TONE_TIMESTAMP_DF[TRIAL_OUTCOME_COL].unique()

array(['rewarded', 'omission', 'win', 'lose'], dtype=object)

- Adding the competition closeness as a column

In [41]:
competition_closeness_map = {k: "non_comp" if "only" in str(k).lower() else "comp" if type(k) is str else np.nan for k in TONE_TIMESTAMP_DF[COMPETITION_CLOSENESS_COL].unique()}

In [42]:
competition_closeness_map

{nan: nan,
 'Subj 1 Only': 'non_comp',
 'Subj 2 blocking Subj 1': 'comp',
 'Subj 1 then Subj 2': 'comp',
 'Subj 1 blocking Subj 2': 'comp',
 'Subj 2 Only': 'non_comp',
 'Subj 2 then Subj 1': 'comp',
 'Close Call': 'comp'}

In [43]:
TONE_TIMESTAMP_DF[COMPETITION_CLOSENESS_COL] = TONE_TIMESTAMP_DF[COMPETITION_CLOSENESS_COL].map(competition_closeness_map)

In [44]:
TONE_TIMESTAMP_DF[COMPETITION_CLOSENESS_COL] = TONE_TIMESTAMP_DF.apply(lambda x: "_".join([str(x[TRIAL_OUTCOME_COL]), str(x[COMPETITION_CLOSENESS_COL])]).strip("nan").strip("_"), axis=1)

In [45]:
TONE_TIMESTAMP_DF[COMPETITION_CLOSENESS_COL].unique()

array(['rewarded', 'omission', 'win_non_comp', 'win_comp',
       'lose_non_comp', 'lose_comp'], dtype=object)

- Removing unnecessary columns

In [51]:
TONE_TIMESTAMP_DF = TONE_TIMESTAMP_DF.drop(columns=[STATE_COL, DIN_COL, ORIGINAL_TRIAL_COL], errors="ignore")
TONE_TIMESTAMP_DF = TONE_TIMESTAMP_DF.drop(columns=[col for col in TONE_TIMESTAMP_DF.columns if "unnamed" in col.lower()], errors="ignore")

In [52]:
TONE_TIMESTAMP_DF.groupby([COMPETITION_CLOSENESS_COL]).count()

,time,recording_dir,recording_file,time_stamp_index,video_file,video_frame,video_number,subject_info,lfp_index,video_name,baseline_lfp_index_range,trial_lfp_index_range,baseline_ephys_index_range,trial_ephys_index_range,baseline_videoframe_range,trial_videoframe_range,all_subjects,current_subject,trial_outcome
competition_closeness,,,,,,,,,,,,,,,,,,,
lose_comp,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105,105
lose_non_comp,89,89,89,89,89,89,89,89,89,89,89,89,89,89,89,89,89,89,89
omission,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48,48
rewarded,290,290,290,290,290,290,290,290,290,290,290,290,290,290,290,290,290,290,290
win_comp,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77
win_non_comp,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134,134


In [53]:
TONE_TIMESTAMP_DF.head()

,time,recording_dir,recording_file,time_stamp_index,video_file,video_frame,video_number,subject_info,competition_closeness,lfp_index,video_name,baseline_lfp_index_range,trial_lfp_index_range,baseline_ephys_index_range,trial_ephys_index_range,baseline_videoframe_range,trial_videoframe_range,all_subjects,current_subject,trial_outcome
0,6310663,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,1390826,20221202_134600_omission_and_competition_subje...,1734,1.0,6_1_top_2_base_3,rewarded,69541,20221202_134600_omission_and_competition_subje...,"(59541, 69541)","(69541, 79541)","(1190826, 1390826)","(1390826, 1590826)","(1514, 1734)","(1734, 1954)","(6.1, 6.2)",6.1,rewarded
1,7910662,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,2990825,20221202_134600_omission_and_competition_subje...,3728,1.0,6_1_top_2_base_3,rewarded,149541,20221202_134600_omission_and_competition_subje...,"(139541, 149541)","(149541, 159541)","(2790825, 2990825)","(2990825, 3190825)","(3508, 3728)","(3728, 3948)","(6.1, 6.2)",6.1,rewarded
2,9710660,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,4790823,20221202_134600_omission_and_competition_subje...,5972,1.0,6_1_top_2_base_3,rewarded,239541,20221202_134600_omission_and_competition_subje...,"(229541, 239541)","(239541, 249541)","(4590823, 4790823)","(4790823, 4990823)","(5752, 5972)","(5972, 6192)","(6.1, 6.2)",6.1,rewarded
3,11310658,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,6390821,20221202_134600_omission_and_competition_subje...,7966,1.0,6_1_top_2_base_3,omission,319541,20221202_134600_omission_and_competition_subje...,"(309541, 319541)","(319541, 329541)","(6190821, 6390821)","(6390821, 6590821)","(7746, 7966)","(7966, 8186)","(6.1, 6.2)",6.1,omission
4,12810657,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,7890820,20221202_134600_omission_and_competition_subje...,9836,1.0,6_1_top_2_base_3,rewarded,394541,20221202_134600_omission_and_competition_subje...,"(384541, 394541)","(394541, 404541)","(7690820, 7890820)","(7890820, 8090820)","(9616, 9836)","(9836, 10056)","(6.1, 6.2)",6.1,rewarded


In [56]:
TONE_TIMESTAMP_DF.to_csv(os.path.join(OUTPUT_DIR, TONE_TIMESTAMPS_CSV))
TONE_TIMESTAMP_DF.to_pickle(os.path.join(OUTPUT_DIR, TONE_TIMESTAMPS_PKL))